In [32]:
import csv
import math
from random import randint, uniform

class CipherSolver:
    def __init__(self):
        # Genetic Algorithm Parameters
        self.num_generations = 500
        self.population_size = 500
        self.tournament_size = 20
        self.tournament_probability = 0.75
        self.crossover_rate = 0.65
        self.num_crossover_points = 5
        self.mutation_rate = 0.2
        self.elitism_rate = 0.15
        self.selection_type = 'TS'
        self.stagnation_limit = 100

        # Weight parameters
        self.bigram_weight = 0.0
        self.trigram_weight = 1.0

        # Usage parameters
        self.verbose_mode = False

        # Default variables
        self.alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
        self.num_elites = int(self.elitism_rate * self.population_size)
        self.num_offsprings = self.population_size - self.num_elites

        self.tournament_probabilities = [self.tournament_probability]

        for i in range(1, self.tournament_size):
            probability = self.tournament_probabilities[i - 1] * (1.0 - self.tournament_probability)
            self.tournament_probabilities.append(probability)

    def load_ngram_frequencies(self, filename):
        ngram_frequencies = {}
        
        with open(filename, 'r') as file:
            reader = csv.reader(file)

            header_skipped = False
            for key, value in reader:
                if not header_skipped:
                    header_skipped = True
                    continue
                
                ngram_frequencies[key] = value

        return ngram_frequencies

    def extract_ngrams(self, text, n):
        ngrams = [text[i:i + n] for i in range(len(text) - n + 1)]

        valid_ngrams = []
        for ngram in ngrams:
            if ngram.isalpha():
                ngram_upper = ngram.upper()
                valid_ngrams.append(ngram_upper)

        return valid_ngrams

    def decrypt_with_key(self, key):
        letter_map = {}

        for i in range(26):
            k = self.alphabet[i]
            v = key.upper()[i]

            letter_map[k] = v

        decrypted_text = ''
        for char in self.cipher_text:
            if char not in self.alphabet:
                decrypted_text += char
            else:
                decrypted_text += letter_map[char]

        return decrypted_text

    def compute_fitness(self, text):
        bigrams = self.extract_ngrams(text, 2)
        trigrams = self.extract_ngrams(text, 3)
        
        bigram_score = 0
        if self.bigram_weight > 0:
            for bigram in bigrams:
                if bigram in self.bigram_frequencies:
                    frequency = int(self.bigram_frequencies[bigram])
                    bigram_score += math.log2(frequency)
        
        trigram_score = 0
        if self.trigram_weight > 0:    
            for trigram in trigrams:
                if trigram in self.trigram_frequencies:
                    frequency = int(self.trigram_frequencies[trigram])
                    trigram_score += math.log2(frequency)
            
        fitness_score = (bigram_score * self.bigram_weight) + (trigram_score * self.trigram_weight)
        return fitness_score

    def combine_keys(self, key1, key2):
        offspring = [None] * 26
        count = 0
        while count < self.num_crossover_points:
            r = randint(0, len(key1) - 1)

            if offspring[r] is None:
                offspring[r] = key1[r]
                count += 1
        
        for char in key2:
            if char not in offspring:
                for i in range(len(offspring)):
                    if offspring[i] is None:
                        offspring[i] = char
                        break

        return ''.join(offspring)

    def mutate_key(self, key):
        a = randint(0, len(key) - 1)
        b = randint(0, len(key) - 1)

        key = list(key)
        temp = key[a]
        key[a] = key[b]
        key[b] = temp

        return ''.join(key)

    def initialize_population(self):
        population = []
        
        for _ in range(self.population_size):
            key = ''

            while len(key) < 26:
                r = randint(0, len(self.alphabet) - 1)
                
                if self.alphabet[r] not in key:
                    key += self.alphabet[r]

            population.append(key)
        
        return population

    def evaluate_population(self, population):
        fitness_scores = []

        for key in population:
            decrypted_text = self.decrypt_with_key(key)
            key_fitness = self.compute_fitness(decrypted_text)
            fitness_scores.append(key_fitness)

        return fitness_scores

    def apply_elitism(self, population, fitness_scores):
        population_fitness = {}
        
        for i in range(self.population_size):
            key = population[i]
            value = fitness_scores[i]

            population_fitness[key] = value

        population_fitness = {k: v for k, v in sorted(population_fitness.items(), key=lambda item: item[1])}
        sorted_population = list(population_fitness.keys())

        elite_population = sorted_population[-self.num_elites:]

        return elite_population

    def roulette_wheel_select(self, fitness_scores):
        index = -1
        max_probability = max(fitness_scores)
            
        selected = False
        while not selected:
            index = randint(0, self.population_size - 1)
            probability = fitness_scores[index]

            r = uniform(0, max_probability)
            selected = (r < probability)
        
        return index

    def select_via_tournament(self, population, fitness_scores):
        remaining_population = population.copy()
        selected_keys = []

        for _ in range(2):
            tournament = {}
            
            for _ in range(self.tournament_size):
                r = randint(0, len(remaining_population) - 1)
                key = remaining_population[r]
                key_fitness = fitness_scores[r]

                tournament[key] = key_fitness
                remaining_population.pop(r)

            sorted_tournament = {k: v for k, v in sorted(tournament.items(), key=lambda item: item[1], reverse=True)}
            tournament_keys = list(sorted_tournament.keys())
            
            index = -1
            selected = False
            while not selected:
                index = randint(0, self.tournament_size - 1)
                probability = self.tournament_probabilities[index]

                r = uniform(0, self.tournament_probability)
                selected = (r < probability)
        
            selected_keys.append(tournament_keys[index])

        return selected_keys[0], selected_keys[1]

    def reproduce_population(self, population, fitness_scores):
        new_population = []

        while len(new_population) < self.num_offsprings:
            parent1, parent2 = None, None

            if self.selection_type == 'RWS':
                parent1_index = self.roulette_wheel_select(fitness_scores)
                parent2_index = self.roulette_wheel_select(fitness_scores)

                parent1 = population[parent1_index]
                parent2 = population[parent2_index]
            else:
                parent1, parent2 = self.select_via_tournament(population, fitness_scores)

            offspring1 = self.combine_keys(parent1, parent2)
            offspring2 = self.combine_keys(parent2, parent1)

            new_population += [offspring1, offspring2]
        
        new_population = self.mutate_population(new_population, self.num_offsprings)

        return new_population
        
    def mutate_population(self, population, pop_size):
        for i in range(pop_size):
            r = uniform(0, 1)

            if r < self.mutation_rate:
                key = population[i]
                mutated_key = self.mutate_key(key)

                population[i] = mutated_key

        return population

    def format_to_plaintext(self, decrypted_text):
        plaintext = list(decrypted_text)
        for i in range(len(plaintext)):
            if self.case_flags[i]:
                plaintext[i] = plaintext[i].lower()
        plaintext = ''.join(plaintext)
        
        return plaintext

    def solve(self, cipher_text = ''):
        # Defining cipher_text
        self.cipher_text = cipher_text
        
        # Checking if cipher_text is valid
        if self.cipher_text == '':
            message = (
                '\n(CipherSolver) Cipher text is invalid. Use solve() as such:\n'
                '\tsolver = CipherSolver()\n'
                '\tsolver.solve("Example cipher text")'
            )
            print(message)
            return

        # Formatting cipher_text
        self.case_flags = [ch.islower() and ch.isalpha() for ch in self.cipher_text]
        self.cipher_text = self.cipher_text.upper()

        # Loading pre-computed ngram frequencies
        bigram_filename = 'data/bi-ngramFrequency.csv'
        self.bigram_frequencies = self.load_ngram_frequencies(bigram_filename)

        trigram_filename = 'data/tri-ngramFrequency.csv'
        self.trigram_frequencies = self.load_ngram_frequencies(trigram_filename)
        
        # Main Program
        population = self.initialize_population()
        
        max_fitness = 0
        stagnation_counter = 0
        for gen_no in range(self.num_generations + 1):
            # Evolving population
            fitness_scores = self.evaluate_population(population)

            # Extracting top solutions
            elite_population = self.apply_elitism(population, fitness_scores)
            reproduction_population = self.reproduce_population(population, fitness_scores)
            population = elite_population + reproduction_population

            # Evaluating population
            max_population_fitness = max(fitness_scores)
            if max_population_fitness > max_fitness:
                max_fitness = max_population_fitness
                stagnation_counter = 0
            else:
                stagnation_counter += 1
            
            index = fitness_scores.index(max_fitness)
            key = population[index]
            decrypted_text = self.decrypt_with_key(key)
            # Verbose Logging
            if self.verbose_mode:
                plaintext = self.format_to_plaintext(decrypted_text)  
                print(f'[INFO] Best fitness (Gen-{gen_no}):', max_fitness)
                print('Key:', key)
                print('Decrypted Text:\n' + plaintext + '\n')
                
            # Checking if stagnated
            if stagnation_counter >= self.stagnation_limit:
                if self.verbose_mode:
                    print('[INFO] Program converged. Stopping execution...\n')
                break
        # Returning plaintext
              
        # Logging final results
        max_population_fitness = max(fitness_scores)
        index = fitness_scores.index(max_population_fitness)
        best_key = population[index]
        decrypted_text = self.decrypt_with_key(best_key)

            
        return plaintext


In [33]:
# Initializing GeneticSolver object
solver = CipherSolver()

# Changing parameters in the GeneticSolver object, if needed.
solver.verbose_mode = True

ciphertext = "snabnxhgzxgmtgxghzazymgvzemwigkfzkzmgagkexvgvzxigkvuzhwkpnzmwwwrfgnacbhvwmbkvuzhbcbatgmgkeebhgvvwmtuwtgxagnkhubkfgxzmbzxwrjwabvbhgagkexwhbgamzrwmiktuzkuztgxgxgxgxbkgveydgfmwnjwrkwyazkbkvuzkzkgvzuwnxzwkvuzbezxwrigmhuuzbxwkzwrvuzigswmrbfnmzkwrhagxbxbhgagkvbnpbvdhgzxgmhugkfzevuzhwnwmzwrvuzubxvwmdwrvuzfmzhwmwigktwmaeezhbcbzadgkebmmzczmxybadvuzfmzhwmwigkxwhbzvdugxyzzkzovbkhvrwmxwawkfvugviwxvwrvuzkgizxwrbvxfmzgvizkizgkabvvazvwvuzgczmgfzzenhgveziwezmkjzmxwkynvhgzxgmxkgizablzgazogkezmxbxxvbaawkjzwjazxabjxvumwnfuwnvvuzhumbxvbgkgkebxagibhtwmaexzczkjzwjaztuwlkwtkwvubkfwrhgzxgmgxgubxvwmbhjzmxwkgabvdgmzrgibabgmtbvuubxrgibadkgizgxgvbvazxbfkbrdbkfgmnazmtuwbxbkxwizxzkxznkbpnzadxnjmzizwmjgmgiwnkvvuzizgbkfwrlgbxzmbkfzmigkvxgmbkvuzxagcwkbhagkfgnfzxgkepgdsgmbkvuzagkfgnfzxwrvuzbxagibhtwmae"

# The function solve() requires the ciphertext as an argument, and returns the plaintext.
plaintext = solver.solve(ciphertext)
print(plaintext)

[INFO] Best fitness (Gen-0): 1284.649798074016
Key: VRCWUDOYMPZKIAGEFSJXTLQBHN
Decrypted Text:
javrabyonboixoboynvnhiolnuiqmozdnzniovozublolnbmozltnyqzeaniqqqsdoavcrylqirzltnyrcrvxoiozuuryollqixtqxobvoazytrzdobnirnbqspqvrlryovozubqyrovinsqimzxtnztnxobobobobrzoluhwodiqapqszqhvnzrzltnznzolntqabnqzltnrunbqsmoiyttnrbqznqsltnmojqisrdainzqsyvobrbryovozlraerlwyonboiytozdnultnyqaqinqsltntrblqiwqsltndinyqiqmozxqivuunyrcrnvwozuriincnibhrvwltndinyqiqmozbqyrnlwtobhnnznglrzylsqibqvqzdltolmqblqsltnzomnbqsrlbdinolmnzmnozvrllvnlqltnocniodnnuayolunmqunizpnibqzhalyonboibzomnvrknovngozunibrbblrvvqzpnqpvnbvrpbltiqadtqalltnytirblrozozurbvomryxqivubncnzpnqpvnxtqkzqxzqltrzdqsyonboiobotrblqirypnibqzovrlwoinsomrvroixrlttrbsomrvwzomnobolrlvnbrdzrswrzdoiavnixtqrbrzbqmnbnzbnazreanvwbapinmnqipoiomqazlltnmnorzdqskorbnirzdnimozlboirzltnbvocqzryvozdoadnbozueowjoirzltnvozdoadnbqsltnrbvomryxqivu

[INFO] Best fitness (Gen-1): 1284.649798074016
Key: RCZOMLSNFYAGWQUJVHBKDIXPTE
Decrypted Text:
bqrcqpnsepswkspsneretwsiemwxf